
<a href="https://colab.research.google.com/github/isikus/MachineScorning/blob/main/Нагенерировать%20машинного%20отупения%20на%20ruGPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Нагенерировать машинного отупения на ruGPT3

Получить свой бредовый текст, прямо как в канале [Машинное отупение](https://t.me/MachineScorning)!

In [1]:
import os
import json

def is_empty(fpath, default_lines=[["Дополни эту фразу", "И эту фразу тоже дополни"], ["Первый пример запроса", "Из нескольких строк", "", "И второй пример", "Я тоже придумать смог"]]):
  if not os.path.isfile(fpath):
    return True
  if not (os.path.getsize(fpath) > 0):
    return True
  if default_lines:
    with open(fpath, "r") as infile:
      lines = infile.read().split("\n")
    for deflines in default_lines:
      if lines == deflines:
        return True
  return False
  

def get_lbc(model_meta):
  lbc = model_meta["line_break_character"]
  lbc = lbc.replace("\s", " ")
  return lbc

#@markdown Для начала выберите модель:

Модель = "Цитаты простых нейролюдей"  #@param ["Нейроописания товаров", "Нейроинструкции на все случаи жизни", "Нейротексты «Короля и Шута»", "Нейроновости Северной Кореи", "Нейропоздравления с новым годом", "Нейроюморески", "Нейросвояк", "Цитаты простых нейролюдей"]

desc_dict = {
    "Нейроописания товаров": "products_desc",
    "Нейроинструкции на все случаи жизни": "wikihowto",
    "Нейротексты «Короля и Шута»": "kishverses",
    "Нейроновости Северной Кореи": "rukcna",
    "Нейропоздравления с новым годом": "greetings",
    "Нейроюморески": "neurojumoreski",
    "Нейросвояк": "neurosvoyak",
    "Цитаты простых нейролюдей": "citati_prosto"
}

model = desc_dict[Модель]

!wget -q --show-progress https://storage.googleapis.com/isikus/machine-scorning/{model}.tar.gz
!mkdir {model}
!tar -xzf {model}.tar.gz -C {model}

try:
  with open(model+"/meta.json", "r") as injson:
    model_meta = json.load(injson)
  print('\x1b[1m'+Модель + ": модель подготовлена"+'\x1b[0m')
except Exception as e:
  print('\x1b[1;31m'+"Что-то сломалось – модель не готова :("+'\x1b[0m')

citati_prosto.tar.g 100%[===================>] 443.69M   137MB/s    in 3.2s    
Цитаты простых нейролюдей: модель подготовлена


Теперь запишите варианты начала сообщений, которые должна дополнить модель, сразу под текстом `%%writefile input.txt`, по 1 разному началу на строку:

In [2]:
%%writefile input.txt
Дополни эту фразу
И эту фразу тоже дополни

Writing input.txt


**Важно:** если ваши запросы содержат переносы строк (например, это начало стихотворения), вместо предыдущей клетки, заполните клетку ниже, разбивая ваш ввод двумя переносами строк. Код ниже заменит переносы строк на специальный символ – это необходимо для корректной работы модели.

In [3]:
%%writefile input_with_lines.txt
Первый пример запроса
Из нескольких строк

И второй пример
Я тоже придумать смог

Writing input_with_lines.txt


Если у вас совсем нет идей, запустите код ниже – он скачает примеры запросов по умолчанию для вашей модели:

In [4]:
if is_empty("input.txt") and is_empty("input_with_lines.txt"):
  !cp {model}/example_input.txt input.txt
  print('\x1b[1;31m'+"Загружены примеры по умолчанию"+'\x1b[0m')

Загружены примеры по умолчанию


In [5]:
#@markdown Наконец, укажите, сколько вариантов выдачи нужно сгенерировать модели на каждый из запросов (от 1 до 30):

Повторов =   3#@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Это маловато. Установили значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Это многовато. Установили значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так. Установили значение " + str(takes) + ".")

## Подготавливаем среду для модели
Сами файлы скачаны, но нужно ещё выполнить код, чтобы подготовить модель к работе.

Проверяем, что мы подключены к видеокарте: команда в клетке ниже должна работать и показывать показывать примерно 12000-15000 Мб свободной памяти:

In [6]:
!nvidia-smi

Sat Sep 11 08:37:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Устанавливаем нужные версии библиотек:

In [7]:
!pip3 install -q urllib3==1.25.4 transformers==2.8.0

print('\n'+'\x1b[1m'+"Требуемые версии пакетов установлены"+'\x1b[0m')

     |████████████████████████████████| 125 kB 5.3 MB/s 
     |████████████████████████████████| 563 kB 38.2 MB/s 
     |████████████████████████████████| 1.2 MB 31.2 MB/s 
     |████████████████████████████████| 5.6 MB 44.1 MB/s 
     |████████████████████████████████| 131 kB 45.9 MB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 7.9 MB 36.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.

Требуемые версии пакетов установлены


Скачиваем файл с исходным кодом для работы модели:

In [8]:
!wget -q --show-progress https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

print('\n'+'\x1b[1m'+"Код запуска модели загружен"+'\x1b[0m')

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

Код запуска модели загружен


Запускаем код, необходимый для преобразования файлов и отображения выдачи:

In [9]:
import re
import os
import sys
import json
import random
from datetime import datetime

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

with open(model+"/meta.json", "r") as injson:
  model_meta = json.load(injson)

# converting inputs

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    isp = intxt.read().split("\n")
    prompts = [p for p in isp if p]
    if not prompts:
      prompts = [p for p in isp]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

print('\x1b[1m'+"Среда подготовлена"+'\x1b[0m')

Среда подготовлена


## Генерируем отупение

Теперь всё, наконец, готово к тому, чтобы модель сгенерировала нам выдачу. То, насколько серьёзный и осмысленный текст будет пытаться выдавать модель, можно регулировать по шкале **`Бредовость`**: здесь есть 5 степеней – от самой нормальной до самой странной (я предупреждал). Кроме этого, флажок **`Повторяться`** определяет, как будет работать модель: если галочку оставить, то первый результат будет очень похож не те, что есть в канале, а последующие будут с ним совпадать. А если снять галочку, то выдача модели каждый раз будет новая. Удачи!

In [10]:
#@markdown ### Начать генерацию!

Бредовость = 0 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

stup_cases = {
    0: {"k": 5, "p": 0.95, "temp": 1.0},
    25: {"k": 5, "p": 0.93, "temp": 2.0},
    50: {"k": 6, "p": 0.91, "temp": 3.0},
    75: {"k": 6, "p": 0.9, "temp": 4.0},
    100: {"k": 7, "p": 0.88, "temp": 5.0}
}

Повторяться = False   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.sub(r"%[a-z]+", "(.*)", re.escape(model_meta["prompt"])), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("<b>" + p + "</b>" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  if Модель == "Нейросвояк":
    rs = re.sub(r"\s([0-9]0?)\.", r"<br><br>\1\\.", rs)
    rs = rs.replace("1. ", "1\. ")
    rs = re.sub(r"\s((?:Ответ: )|(?:Источник[а-яё()]: ))", r"<br>\1", rs)
    sq = re.search(r"Автор[а-яё()]?: ", rs)
    if sq:
      rs = rs[:sq.start() - 1]
  rse = len(rs) if rs.find("Context >>") == -1 else rs.find("Context >>")
  printmd(rs[:rse] + "<br>")

Ключ случайного значения: **8003**

Ты мне скажи, я тебе скажу<br>

Ну как, как? Всё нормально?<br>

Я так и знала<br>

А что, у нас уже в этом месяце?<br>

...

Да, это точно, я в курсе.  И что, все довольны?<br>

А ты не обижайся<br>

Да, да, да, конечно, конечно, это всё, что я хотел сказать. Но я просто хотел сказать, что я всё ещё очень рад, что всё получилось, всё очень понравилось. И всё-таки, я думаю, что я всё-таки ещё не дошёл до этого. И, вообще-то, я не думаю, что это всё, что я хочу сказать. Просто я очень рад, что всё сложилось так, как я хочу. И вообще, я просто не понимаю, почему я так делаю. И вообще, мне кажется, что это очень круто. Это как-то само собой происходит. Всё-таки я не знаю, как всё должно получиться. Я бы не хотел, чтобы всё получилось так, как я хочу. И ещё мне кажется, что всё будет хорошо. Я не хочу быть в этом плане счастливым, просто я хочу, чтобы всё было как в жизни. И всё-таки я рад, что всё получилось, всё было так, как я хочу. И я очень благодарен тебе за всё, что ты для меня сделал, что я тебе сделал, что я для тебя сделал. Спасибо, что ты есть. Спасибо, что ты есть. Я тебя люблю, очень.

В Москве в пятницу, 14 февраля, на Пушкинской площади пройдёт фестиваль &quot;Парад планет&quot;, в рамках которого пройдут соревнования по плаванию на воде. В программе - бег, плавание, катание на коньках, беговые лыжи, волейбол, волейбол, плавание, плавание, волейбол, волейбол, футбол.

В Москве в субботу, 14 февраля, на Пушкинской площади пройдет фестиваль &quot;Парад планет&quot;, в рамках которого пройдут соревнования по плаванию на воде. В программе - бег, плавание, плавание, катание на коньках, плавание, плавание, волейбол, волейбол, плавание, плавание, волейбол, футбол.

В Москве в воскресенье, 15 февраля, на Пушкинской площади пройдёт фестиваль &quot;Парад планет&quot;, в рамках которого пройдут соревнования по плаванию на воде. В программе - бег, плавание, плавание, плавание, плавание, плавание, волейбол, волейбол, плавание, волейбол, плавание, волейбол, плавание, волейбол, волейбол, плавание, волейбол, плавание, волейбол, плавание, волейбол, плавание, волейбол, волейбол, плавание, волейбол, плавание, волейбол, плавание, волейбол, плавание, волейбо<br>

Да я уже и забыл, где ты был<br>

Я тебя люблю!  Ты мне нравишься 
<br>

## Больше отупения!

Хотите попробовать другие фразы? Тогда впишите их в клетку под этим текстом, как до этого, и запустите код ниже. Не забудьте поменять `%%writefile input.txt` на `%%writefile input_with_lines.txt`, если ваши запросы содержат переносы строк!

In [11]:
%%writefile input.txt






Overwriting input.txt


In [22]:
#@markdown ### Генерируем новую выдачу

Повторов = 3  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Многовато повторов. Установлено значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так с количеством повторов. Установлено значение " + str(takes) + ".")

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    isp = intxt.read().split("\n")
    prompts = [p for p in isp if p]
    if not prompts:
      prompts = [p for p in isp]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

Бредовость = 25 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

Повторяться = False   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.sub(r"%[a-z]+", "(.*)", re.escape(model_meta["prompt"])), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("<b>" + p + "</b>" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  if Модель == "Нейросвояк":
    rs = re.sub(r"\s([0-9]0?)\.", r"<br><br>\1\\.", rs)
    rs = rs.replace("1. ", "1\. ")
    rs = re.sub(r"\s((?:Ответ: )|(?:Источник[а-яё()]: ))", r"<br>\1", rs)
    sq = re.search(r"Автор[а-яё()]?: ", rs)
    if sq:
      rs = rs[:sq.start() - 1]
  rse = len(rs) if rs.find("Context >>") == -1 else rs.find("Context >>")
  printmd(rs[:rse] + "<br>")

Ключ случайного значения: **4432**

А что за фильм?
- Да там вообще-то про любовь, - говорит он и смотрит на нас<br>

Ты не расстраивайся, всё будет хорошо)<br>

А у тебя?<br>

Я не хочу быть в этом мире ~draggle<br>

...

Я просто не понимаю как так получилось, что у вас в офисе такая огромная очередь, я не успеваю за вами и вы опаздываете на поезд, да еще с опозданием<br>

Ну, я думаю, ты уже понял что-нибудь.<br>

Я тоже не знаю, может ты и прав.  А я вот не знаю...  Может ты не права и мне надо идти..? ~frr~<br>

Ну, как там наши поживают?<br>

Ты мне нравишься 
<br>

На этом всё. Получайте удовольствие (надеюсь)!